 1. Xử lí data từ json

In [1]:
#Import thư viện cần thiết
import json
import pandas as pd
from itertools import zip_longest
import numpy as np
import torch
import re
from sentence_transformers import SentenceTransformer, util
from tqdm import tqdm
import os
from sacrebleu.metrics import BLEU
from bert_score import score
from googletrans import Translator
import time

c:\Users\DELL\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
rows = []

# Đọc file JSON
with open('../data/json/JSON1.json', 'r', encoding='utf-8') as f:
    data = json.load(f)

# Lọc id trong khoảng from_id–to_id
from_id = 1061
to_id = 1272
data = [item for item in data if from_id <= item.get('id', -1) <= to_id]

# Xử lý từng item
for item in data:
    base_id = item.get('id')
    cn_text = item.get('cn', '')
    vi_text = item.get('vi', '')

    # Tách dòng
    cn_lines = cn_text.split('\n')
    vi_lines = vi_text.split('\n')

    line_number = 1  # counter cho mỗi id

    for cn, vi in zip_longest(cn_lines, vi_lines, fillvalue=''):
        rows.append({
            'src_id': f"{base_id}",
            'src_lang': cn,
            'tgt_lang': vi
        })

# Xuất thành DataFrame
df = pd.DataFrame(rows)

In [3]:
df.info()
df.head(10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13724 entries, 0 to 13723
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   src_id    13724 non-null  object
 1   src_lang  13724 non-null  object
 2   tgt_lang  13724 non-null  object
dtypes: object(3)
memory usage: 321.8+ KB


,src_id,src_lang,tgt_lang
0,1061,MAC地址的全称是媒体访问控制地址，每个MAC地址都是与众不同的，可被用来识别网络中不同电脑...,Đôi khi bạn muốn thay đổi địa chỉ MAC của bộ đ...
1,1061,步骤,Các bước
2,1061,方法 1,Phương pháp 1
3,1061,方法 1 的 3:,Phương pháp 1 của 2:
4,1061,使用设备管理器,Sử dụng Device Manager (Trình quản lý Thiết bị)
5,1061,-,-
6,1061,1. 打开设备管理器。你可以从控制面板中进入设备管理器。如果你使用的是分类视图，那么它位于系...,1. Mở Device Manager. Bạn có thể truy cập Devi...
7,1061,-,-
8,1061,2. 打开设备管理器，你将看到列有安装在电脑里的所有硬件的列表。所有硬件按类别分类排列。点击...,2. Mở rộng mục Network Adapters (Bộ điều hợp M...
9,1061,- 如果你不确定你使用的是哪个适配器，那么查看本文开头来找到关于你设备的描述。,- Nếu không dám chắc mình đang sử dụng bộ điều...


In [4]:
# Xóa các dòng chứa kí tự "-,-", ",-" hoặc "-,"
remove_mask = (
    (df["src_lang"] == "-") |
    (df["tgt_lang"] == "-") |
    (df["src_lang"] + "," + df["tgt_lang"]).isin(["-,-", ",-", "-,"])
)

df_clean = df[~remove_mask].copy()
df_clean = df_clean.reset_index(drop=True)

# Xuất CSV
df_clean.to_csv(f"../data/preprocessing_data/json1_{from_id}_{to_id}.csv", index=False, encoding="utf-8-sig")

df_clean.info()
df_clean.head(10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9309 entries, 0 to 9308
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   src_id    9309 non-null   object
 1   src_lang  9309 non-null   object
 2   tgt_lang  9309 non-null   object
dtypes: object(3)
memory usage: 218.3+ KB


,src_id,src_lang,tgt_lang
0,1061,MAC地址的全称是媒体访问控制地址，每个MAC地址都是与众不同的，可被用来识别网络中不同电脑...,Đôi khi bạn muốn thay đổi địa chỉ MAC của bộ đ...
1,1061,步骤,Các bước
2,1061,方法 1,Phương pháp 1
3,1061,方法 1 的 3:,Phương pháp 1 của 2:
4,1061,使用设备管理器,Sử dụng Device Manager (Trình quản lý Thiết bị)
5,1061,1. 打开设备管理器。你可以从控制面板中进入设备管理器。如果你使用的是分类视图，那么它位于系...,1. Mở Device Manager. Bạn có thể truy cập Devi...
6,1061,2. 打开设备管理器，你将看到列有安装在电脑里的所有硬件的列表。所有硬件按类别分类排列。点击...,2. Mở rộng mục Network Adapters (Bộ điều hợp M...
7,1061,- 如果你不确定你使用的是哪个适配器，那么查看本文开头来找到关于你设备的描述。,- Nếu không dám chắc mình đang sử dụng bộ điều...
8,1061,3. 右键点击你的适配器。从菜单中点击属性来打开网络适配器的属性窗口。,3. Nhấp chuột phải vào bộ điều hợp. Chọn Prope...
9,1061,4. 点击高级选项卡。找到“网络地址”或“本地管理地址”条目。点击该条目，你将在右侧看到“值...,4. Nhấp chuột vào tab Advanced (Nâng cao). Tìm...


In [ ]:
rows = []

# Đọc file JSON
with open('../data/json/JSON2.json', 'r', encoding='utf-8') as f:
    data = json.load(f)

# Lọc id trong khoảng from_id–to_id
from_id = 5816
to_id = 6978
data = [item for item in data if from_id <= item.get('id', -1) <= to_id]


# Xử lý từng item
for item in data:
    base_id = item.get('id')
    zh_text = item.get('zh', '')
    vi_text = item.get('vi', '')

    # Tách dòng
    zh_lines = zh_text.split('\n')
    vi_lines = vi_text.split('\n')

    line_number = 1  # counter cho mỗi id

    for zh, vi in zip_longest(zh_lines, vi_lines, fillvalue=''):
        rows.append({
            'src_id': f"{base_id}",
            'src_lang': zh,
            'tgt_lang': vi
        })

# Xuất thành DataFrame
df = pd.DataFrame(rows)

In [9]:
df.info()
df.head(10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1163 entries, 0 to 1162
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   src_id    1163 non-null   object
 1   src_lang  1163 non-null   object
 2   tgt_lang  1163 non-null   object
dtypes: object(3)
memory usage: 27.4+ KB


,src_id,src_lang,tgt_lang
0,5816,所以我开始帮助他们，不是做日常的运输，而是帮助他们理解如何量化他们需要多少药物,"Vì vậy, tôi đã bắt đầu giúp đỡ với việc đó, kh..."
1,5817,当时我们给予的所有药物都是单独的药物，剂量不同，所以不像一天一次吃一粒药那样简单。,Tất cả các loại thuốc mà chúng tôi đang dùng v...
2,5818,因此，为了帮助他们理解如何提供完整的治疗方案和计划，我做了几年，主要是在这种类型的,"và do đó, để giúp họ hiểu cách cung cấp một ch..."
3,5819,扩大容量，但我在制定培训计划时，不断遇到一些反复出现的临床问题。,"Mở rộng năng lực, nhưng khi tôi làm việc trên ..."
4,5820,其中一个特别的问题是，对于接受逆转录病毒治疗的患者，我能使用什么类型的避孕方法，因为,và một trong số những điều đó đặc biệt là loại...
5,5821,存在大量的药物相互作用，而且大多数与艾滋病毒相关的研究是在美国或欧洲进行的，,Có rất nhiều tương tác thuốc và hầu hết các ng...
6,5822,当时的艾滋病流行病主要影响男性，而且当时的临床试验中女性参与者很少。,"Dịch bệnh HIV, đặc biệt là vào thời điểm đó, c..."
7,5823,关于如何管理女性健康问题的信息并不多，所以这成了我需要自己解决的事情之一。,Không có nhiều thông tin về cách quản lý các v...
8,5824,我只是没有一个好的答案来告诉他们关于哪种有效的避孕方法，比如我可以提供的激素避孕方法,Tôi chỉ không có một câu trả lời tốt cho họ về...
9,5825,除了使用安全套之外，撒哈拉以南非洲的女性并不总是能够强制使用安全套。,"Ngoài việc sử dụng bao cao su, phụ nữ ở châu P..."


In [10]:
# Xóa các dòng chứa kí tự "-,-", ",-" hoặc "-,"
remove_mask = (
    (df["src_lang"] == "-") |
    (df["tgt_lang"] == "-") |
    (df["src_lang"] + "," + df["tgt_lang"]).isin(["-,-", ",-", "-,"])
)

df_clean = df[~remove_mask].copy()
df_clean = df_clean.reset_index(drop=True)

# Ghi đè vào file gốc
df_clean.to_csv(f"../data/preprocessing_data/json2_{from_id}_{to_id}.csv", index=False, encoding="utf-8-sig")

df_clean.info()
df_clean.head(10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1163 entries, 0 to 1162
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   src_id    1163 non-null   object
 1   src_lang  1163 non-null   object
 2   tgt_lang  1163 non-null   object
dtypes: object(3)
memory usage: 27.4+ KB


,src_id,src_lang,tgt_lang
0,5816,所以我开始帮助他们，不是做日常的运输，而是帮助他们理解如何量化他们需要多少药物,"Vì vậy, tôi đã bắt đầu giúp đỡ với việc đó, kh..."
1,5817,当时我们给予的所有药物都是单独的药物，剂量不同，所以不像一天一次吃一粒药那样简单。,Tất cả các loại thuốc mà chúng tôi đang dùng v...
2,5818,因此，为了帮助他们理解如何提供完整的治疗方案和计划，我做了几年，主要是在这种类型的,"và do đó, để giúp họ hiểu cách cung cấp một ch..."
3,5819,扩大容量，但我在制定培训计划时，不断遇到一些反复出现的临床问题。,"Mở rộng năng lực, nhưng khi tôi làm việc trên ..."
4,5820,其中一个特别的问题是，对于接受逆转录病毒治疗的患者，我能使用什么类型的避孕方法，因为,và một trong số những điều đó đặc biệt là loại...
5,5821,存在大量的药物相互作用，而且大多数与艾滋病毒相关的研究是在美国或欧洲进行的，,Có rất nhiều tương tác thuốc và hầu hết các ng...
6,5822,当时的艾滋病流行病主要影响男性，而且当时的临床试验中女性参与者很少。,"Dịch bệnh HIV, đặc biệt là vào thời điểm đó, c..."
7,5823,关于如何管理女性健康问题的信息并不多，所以这成了我需要自己解决的事情之一。,Không có nhiều thông tin về cách quản lý các v...
8,5824,我只是没有一个好的答案来告诉他们关于哪种有效的避孕方法，比如我可以提供的激素避孕方法,Tôi chỉ không có một câu trả lời tốt cho họ về...
9,5825,除了使用安全套之外，撒哈拉以南非洲的女性并不总是能够强制使用安全套。,"Ngoài việc sử dụng bao cao su, phụ nữ ở châu P..."


2. Alignment cho json 

In [2]:
SRC = 'JSON1'
FROM_ID = 1061
TO_ID = 1272
INPUT_FILE = f"../data/preprocessing_data/{SRC.lower()}_{FROM_ID}_{TO_ID}.csv"
OUTPUT_FILE = f"../Alignment/{SRC}_{FROM_ID}_{TO_ID}.csv"
ALIGN_THRESHOLD = 0.75

In [ ]:
def get_device():
    if torch.cuda.is_available():
        return 'cuda'
    return 'cpu'

def split_sentences(text, lang='vi'):
    if not text: return []
    if lang == 'zh':
        sents = re.split(r'(?<=[。！？])\s*', text)
    else:
        sents = re.split(
            r'(?<!\d\.)(?<=[.?!])\s+(?=(?:[A-ZÀ-Ỵ"\'(]|\d{1,3}\.))',
            text
        )

    return [s.strip() for s in sents]

def align_n3(src_sents, tgt_sents, model, device, threshold):
    if not src_sents or not tgt_sents:
        return []

    # Encode
    # 1-gram (Câu đơn)
    emb_src = model.encode(src_sents, convert_to_tensor=True, device=device)
    emb_tgt = model.encode(tgt_sents, convert_to_tensor=True, device=device)

    # 2-gram (Gộp 2 câu)
    tgt_merged_2 = [tgt_sents[i] + " " + tgt_sents[i+1] for i in range(len(tgt_sents)-1)]
    src_merged_2 = [src_sents[i] + src_sents[i+1] for i in range(len(src_sents)-1)]

    emb_tgt_2 = model.encode(tgt_merged_2, convert_to_tensor=True, device=device) if tgt_merged_2 else None
    emb_src_2 = model.encode(src_merged_2, convert_to_tensor=True, device=device) if src_merged_2 else None

    # 3-gram (Gộp 3 câu)
    tgt_merged_3 = [tgt_sents[i] + " " + tgt_sents[i+1] + " " + tgt_sents[i+2] for i in range(len(tgt_sents)-2)]
    src_merged_3 = [src_sents[i] + src_sents[i+1] + src_sents[i+2] for i in range(len(src_sents)-2)]

    emb_tgt_3 = model.encode(tgt_merged_3, convert_to_tensor=True, device=device) if tgt_merged_3 else None
    emb_src_3 = model.encode(src_merged_3, convert_to_tensor=True, device=device) if src_merged_3 else None

    # Tính ma trận tương đồng (similarity matrices)
    # Khởi tạo ma trận (rỗng chiều)
    def get_sim(emb1, emb2):
        if emb1 is not None and emb2 is not None:
            return util.cos_sim(emb1, emb2).cpu().numpy()
        return np.zeros((0,0))

    sim_1_1 = get_sim(emb_src, emb_tgt)

    sim_1_2 = get_sim(emb_src, emb_tgt_2)     # Src đơn vs Tgt gộp 2
    sim_2_1 = get_sim(emb_src_2, emb_tgt)     # Src gộp 2 vs Tgt đơn

    sim_1_3 = get_sim(emb_src, emb_tgt_3)     # Src đơn vs Tgt gộp 3
    sim_3_1 = get_sim(emb_src_3, emb_tgt)     # Src gộp 3 vs Tgt đơn

    results = []

    # Đánh dấu câu đã dùng
    used_src = np.zeros(len(src_sents), dtype=bool)
    used_tgt = np.zeros(len(tgt_sents), dtype=bool)

    # Greedy search
    while True:
        best_score = -1
        best_type = None
        best_indices = None # (r, c)

        # Helper để check max score trong matrix
        def check_matrix(matrix, label):
            nonlocal best_score, best_type, best_indices
            if matrix.size > 0:
                r, c = np.unravel_index(matrix.argmax(), matrix.shape)
                if matrix[r, c] > best_score:
                    best_score = matrix[r, c]
                    best_type = label
                    best_indices = (r, c)

        # Kiểm tra tất cả các trường hợp
        check_matrix(sim_1_1, '1-1')
        check_matrix(sim_1_2, '1-2')
        check_matrix(sim_2_1, '2-1')
        check_matrix(sim_1_3, '1-3')
        check_matrix(sim_3_1, '3-1')

        if best_score < threshold:
            break

        r, c = best_indices
        valid_match = False

        # Logic kết quả và masking
        # Logic chung: Kiểm tra used -> Add result -> Mark used -> Mask matrix

        if best_type == '1-1':
            if not used_src[r] and not used_tgt[c]:
                results.append({'src': src_sents[r], 'tgt': tgt_sents[c], 'type': '1-1'})
                used_src[r] = True; used_tgt[c] = True
                valid_match = True
                # Masking: Xóa hàng r (src) và cột c (tgt) ở mọi matrix liên quan: gán -1 trực tiếp vào các vùng ảnh hưởng
                sim_1_1[r, :] = -1; sim_1_1[:, c] = -1
                if sim_1_2.size: sim_1_2[r, :] = -1; sim_1_2[:, max(0, c-1):c+1] = -1
                if sim_2_1.size: sim_2_1[max(0, r-1):r+1, :] = -1; sim_2_1[:, c] = -1
                if sim_1_3.size: sim_1_3[r, :] = -1; sim_1_3[:, max(0, c-2):c+1] = -1
                if sim_3_1.size: sim_3_1[max(0, r-2):r+1, :] = -1; sim_3_1[:, c] = -1

        elif best_type == '1-2': # Src r vs Tgt c, c+1
            if not used_src[r] and not used_tgt[c] and not used_tgt[c+1]:
                results.append({'src': src_sents[r], 'tgt': tgt_sents[c] + " " + tgt_sents[c+1], 'type': '1-2'})
                used_src[r] = True; used_tgt[c:c+2] = True
                valid_match = True
                sim_1_2[r, c] = -1 # Xóa chính nó
                sim_1_1[r, :] = -1; sim_1_1[:, c:c+2] = -1


        elif best_type == '2-1': # Src r, r+1 vs Tgt c
            if not used_src[r] and not used_src[r+1] and not used_tgt[c]:
                results.append({'src': src_sents[r]+src_sents[r+1], 'tgt': tgt_sents[c], 'type': '2-1'})
                used_src[r:r+2] = True; used_tgt[c] = True
                valid_match = True
                sim_2_1[r, c] = -1
                sim_1_1[r:r+2, :] = -1; sim_1_1[:, c] = -1

        elif best_type == '1-3': # Src r vs Tgt c, c+1, c+2
            if not used_src[r] and not used_tgt[c] and not used_tgt[c+1] and not used_tgt[c+2]:
                results.append({'src': src_sents[r], 'tgt': tgt_sents[c]+" "+tgt_sents[c+1]+" "+tgt_sents[c+2], 'type': '1-3'})
                used_src[r] = True; used_tgt[c:c+3] = True
                valid_match = True
                sim_1_3[r, c] = -1
                sim_1_1[r, :] = -1; sim_1_1[:, c:c+3] = -1

        elif best_type == '3-1': # Src r, r+1, r+2 vs Tgt c
            if not used_src[r] and not used_src[r+1] and not used_src[r+2] and not used_tgt[c]:
                results.append({'src': src_sents[r]+src_sents[r+1]+src_sents[r+2], 'tgt': tgt_sents[c], 'type': '3-1'})
                used_src[r:r+3] = True; used_tgt[c] = True
                valid_match = True
                sim_3_1[r, c] = -1
                sim_1_1[r:r+3, :] = -1; sim_1_1[:, c] = -1

        # Nếu không match được (do conflict đã bị dùng) -> xóa điểm này để tìm max tiếp theo
        if not valid_match:
            if best_type == '1-1': sim_1_1[r, c] = -1
            elif best_type == '1-2': sim_1_2[r, c] = -1
            elif best_type == '2-1': sim_2_1[r, c] = -1
            elif best_type == '1-3': sim_1_3[r, c] = -1
            elif best_type == '3-1': sim_3_1[r, c] = -1

        # Cập nhật kết quả
        if valid_match:
            results[-1]['score'] = float(best_score)

    return results

In [ ]:
device = get_device()
print(f"Đang tải model LaBSE trên {device} (n=3)...")
model = SentenceTransformer('sentence-transformers/LaBSE', device=device)

df = pd.read_csv(INPUT_FILE)
df["src_lang"] = df["src_lang"].fillna("").astype(str)
df["tgt_lang"] = df["tgt_lang"].fillna("").astype(str)

final_data = []

print("Bắt đầu Alignment (1-1, 1-2, 2-1, 1-3, 3-1)...")
for idx, row in tqdm(df.iterrows(), total=len(df)):
    src_sents = split_sentences(row.get('src_lang', ''), lang='zh')
    tgt_sents = split_sentences(row.get('tgt_lang', ''), lang='vi')
    pairs = align_n3(src_sents, tgt_sents, model, device, ALIGN_THRESHOLD)

    for p in pairs:
        final_data.append({
            'src_id': row.get('src_id', ''),
            'src_lang': p['src'],
            'tgt_lang': p['tgt'],
            'score': p['score'],
            'type': p.get('type', '1-1')
        })

# Thống kê báo cáo
df_res = pd.DataFrame(final_data)
type_counts = df_res['type'].value_counts()

print("Kết quả Alignment")
print(f"Tổng số cặp: {len(df_res)}")
print(f"Phân bố cặp câu: {type_counts}")
print(f"Average Semantic Score: {df_res['score'].mean():.4f}")

# Lưu file
df_res[['src_id', 'src_lang', 'tgt_lang']].to_csv(OUTPUT_FILE, index=False, encoding='utf-8-sig')
print(f"Đã lưu file: {OUTPUT_FILE}")

Đang tải model LaBSE trên cpu (n=3)...
Bắt đầu Alignment (1-1, 1-2, 2-1, 1-3, 3-1)...


100%|██████████| 9309/9309 [1:14:36<00:00,  2.08it/s] 

Kết quả Alignment
Tổng số cặp: 6262
Phân bố cặp câu: type
1-1    5150
2-1     794
1-2     207
3-1      91
1-3      20
Name: count, dtype: int64
Average Semantic Score: 0.8571
Đã lưu file: ../Alignment/JSON1_1061_1272.csv


3. Evaluate của Alignment (json1)

In [ ]:
# Config
SRC = 'JSON1'
FROM_ID = 1061
TO_ID = 1272
INPUT_FILE = f"../Alignment/{SRC}_{FROM_ID}_{TO_ID}.csv"
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'

In [ ]:
def get_proxy_translations(df, sample_size):
    """
    Dịch câu nguồn tiếng Trung sang tiếng Việt dùng Google Translate
    để làm tham chiếu so sánh.
    """
    print(f"Đang lấy mẫu ngẫu nhiên {sample_size} cặp câu để đánh giá...")

    # Lấy mẫu ngẫu nhiên
    sample = df.sample(n=min(sample_size, len(df)), random_state=10).copy()

    translator = Translator()
    hypotheses = [] # Câu trong file
    references = [] # Câu dịch máy (Làm chuẩn so sánh)
    success_count = 0

    for idx, row in tqdm(sample.iterrows(), total=len(sample)):
        src = str(row['src_lang'])
        tgt = str(row['tgt_lang']) # Đây là câu tiếng Việt gốc trong dataset

        # Bỏ qua câu quá ngắn hoặc rỗng
        if len(src) < 2 or len(tgt) < 5:
            continue

        try:
            # Dịch Trung -> Việt
            translated = translator.translate(src, src='zh-cn', dest='vi').text

            # Câu Google dịch là Reference (Tham chiếu) và câu Tgt gốc là Hypothesis (Giả thuyết)
            references.append(translated)
            hypotheses.append(tgt)

            success_count += 1
            time.sleep(0.5)

        except Exception as e:
            # Nếu lỗi mạng thì bỏ qua
            continue

    print(f"Đã dịch thành công {success_count} câu.")
    return hypotheses, references

def calculate_metrics(hypotheses, references):
    """Tính toán BLEU và BERTScore"""
    if not hypotheses:
        print("Không có dữ liệu để tính toán.")
        return

    # Tính BLEU Score
    # BLEU đánh giá độ khớp từ vựng (n-gram overlap)
    bleu = BLEU()
    bleu_score = bleu.corpus_score(hypotheses, [references])

    print(f"BLEU Score: {bleu_score.score:.2f}")

    # Tính BERTScore
    # BERTScore đánh giá độ tương đồng ngữ nghĩa (Semantic Similarity) dùng mô hình BERT
    P, R, F1 = score(hypotheses, references, lang='vi', verbose=False, device=DEVICE)

    avg_bert = F1.mean().item()
    print(f"BERTScore (F1): {avg_bert:.4f}")

    print("Ví dụ tham chiếu (Top 3):")
    for i in range(min(3, len(hypotheses))):
        print(f"Cặp {i+1}:")
        print(f"Dataset Tgt: {hypotheses[i]}")
        print(f"Google Trans: {references[i]}")

In [ ]:
df = pd.read_csv(INPUT_FILE)
# Thực hiện quy trình đánh giá
SAMPLE_SIZE = int(df.shape[0] * 0.1)  # Số lượng câu lấy mẫu để đánh giá
hyps, refs = get_proxy_translations(df, SAMPLE_SIZE)
# Tính điểm
calculate_metrics(hyps, refs)

Đang lấy mẫu ngẫu nhiên 626 cặp câu để đánh giá...


100%|██████████| 626/626 [20:33<00:00,  1.97s/it]


Đã dịch thành công 616 câu.
BLEU Score: 33.00
BERTScore (F1): 0.8874
Ví dụ tham chiếu (Top 3):
Cặp 1:
Dataset Tgt: 4. Dùng bàn tay ép mạnh lên dao để làm nát tỏi. Tỏi sẽ dễ dàng bị nghiền do lực ấn từ tay của bạn.
Google Trans: 4. Dùng tay ấn vào mặt sau của dao. Những tép tỏi sẽ dễ dàng bị nghiền nát dưới áp lực của bạn.
Cặp 2:
Dataset Tgt: Lời khuyên
Google Trans: Mẹo
Cặp 3:
Dataset Tgt: 4. Gõ .attrib -h -r -s /s /d X:\*.* và nhấn ↵ Enter.
Google Trans: 4. Nhập .attrib -h -r -s /s /d X:\*.* và nhấn Enter.


5. Alignment cho json 2

In [9]:
SRC = 'JSON2'
FROM_ID = 5816
TO_ID = 6978
INPUT_FILE = f"../data/preprocessing_data/{SRC.lower()}_{FROM_ID}_{TO_ID}.csv"
OUTPUT_FILE = f"../Alignment/{SRC}_{FROM_ID}_{TO_ID}.csv"
ALIGN_THRESHOLD = 0.75

In [ ]:
def get_device():
    if torch.cuda.is_available():
        return 'cuda'
    return 'cpu'

def split_sentences(text, lang='vi'):
    if not text: return []
    if lang == 'zh':
        sents = re.split(r'(?<=[。！？])\s*', text)
    else:
        sents = re.split(
            r'(?<!\d\.)(?<=[.?!])\s+(?=(?:[A-ZÀ-Ỵ"\'(]|\d{1,3}\.))',
            text
        )

    return [s.strip() for s in sents]

def align_n3(src_sents, tgt_sents, model, device, threshold):
    if not src_sents or not tgt_sents:
        return []

    # Encode
    # 1-gram (Câu đơn)
    emb_src = model.encode(src_sents, convert_to_tensor=True, device=device)
    emb_tgt = model.encode(tgt_sents, convert_to_tensor=True, device=device)

    # 2-gram (Gộp 2 câu)
    tgt_merged_2 = [tgt_sents[i] + " " + tgt_sents[i+1] for i in range(len(tgt_sents)-1)]
    src_merged_2 = [src_sents[i] + src_sents[i+1] for i in range(len(src_sents)-1)]

    emb_tgt_2 = model.encode(tgt_merged_2, convert_to_tensor=True, device=device) if tgt_merged_2 else None
    emb_src_2 = model.encode(src_merged_2, convert_to_tensor=True, device=device) if src_merged_2 else None

    # 3-gram (Gộp 3 câu)
    tgt_merged_3 = [tgt_sents[i] + " " + tgt_sents[i+1] + " " + tgt_sents[i+2] for i in range(len(tgt_sents)-2)]
    src_merged_3 = [src_sents[i] + src_sents[i+1] + src_sents[i+2] for i in range(len(src_sents)-2)]

    emb_tgt_3 = model.encode(tgt_merged_3, convert_to_tensor=True, device=device) if tgt_merged_3 else None
    emb_src_3 = model.encode(src_merged_3, convert_to_tensor=True, device=device) if src_merged_3 else None

    # Tính ma trận tương đồng (similarity matrices)
    # Khởi tạo ma trận (rỗng chiều)
    def get_sim(emb1, emb2):
        if emb1 is not None and emb2 is not None:
            return util.cos_sim(emb1, emb2).cpu().numpy()
        return np.zeros((0,0))

    sim_1_1 = get_sim(emb_src, emb_tgt)

    sim_1_2 = get_sim(emb_src, emb_tgt_2)     # Src đơn vs Tgt gộp 2
    sim_2_1 = get_sim(emb_src_2, emb_tgt)     # Src gộp 2 vs Tgt đơn

    sim_1_3 = get_sim(emb_src, emb_tgt_3)     # Src đơn vs Tgt gộp 3
    sim_3_1 = get_sim(emb_src_3, emb_tgt)     # Src gộp 3 vs Tgt đơn

    results = []

    # Đánh dấu câu đã dùng
    used_src = np.zeros(len(src_sents), dtype=bool)
    used_tgt = np.zeros(len(tgt_sents), dtype=bool)

    # Greedy search
    while True:
        best_score = -1
        best_type = None
        best_indices = None # (r, c)

        # Helper để check max score trong matrix
        def check_matrix(matrix, label):
            nonlocal best_score, best_type, best_indices
            if matrix.size > 0:
                r, c = np.unravel_index(matrix.argmax(), matrix.shape)
                if matrix[r, c] > best_score:
                    best_score = matrix[r, c]
                    best_type = label
                    best_indices = (r, c)

        # Kiểm tra tất cả các trường hợp
        check_matrix(sim_1_1, '1-1')
        check_matrix(sim_1_2, '1-2')
        check_matrix(sim_2_1, '2-1')
        check_matrix(sim_1_3, '1-3')
        check_matrix(sim_3_1, '3-1')

        if best_score < threshold:
            break

        r, c = best_indices
        valid_match = False

        # Logic kết quả và masking
        # Logic chung: Kiểm tra used -> Add result -> Mark used -> Mask matrix

        if best_type == '1-1':
            if not used_src[r] and not used_tgt[c]:
                results.append({'src': src_sents[r], 'tgt': tgt_sents[c], 'type': '1-1'})
                used_src[r] = True; used_tgt[c] = True
                valid_match = True
                # Masking: Xóa hàng r (src) và cột c (tgt) ở mọi matrix liên quan: gán -1 trực tiếp vào các vùng ảnh hưởng
                sim_1_1[r, :] = -1; sim_1_1[:, c] = -1
                if sim_1_2.size: sim_1_2[r, :] = -1; sim_1_2[:, max(0, c-1):c+1] = -1
                if sim_2_1.size: sim_2_1[max(0, r-1):r+1, :] = -1; sim_2_1[:, c] = -1
                if sim_1_3.size: sim_1_3[r, :] = -1; sim_1_3[:, max(0, c-2):c+1] = -1
                if sim_3_1.size: sim_3_1[max(0, r-2):r+1, :] = -1; sim_3_1[:, c] = -1

        elif best_type == '1-2': # Src r vs Tgt c, c+1
            if not used_src[r] and not used_tgt[c] and not used_tgt[c+1]:
                results.append({'src': src_sents[r], 'tgt': tgt_sents[c] + " " + tgt_sents[c+1], 'type': '1-2'})
                used_src[r] = True; used_tgt[c:c+2] = True
                valid_match = True
                sim_1_2[r, c] = -1 # Xóa chính nó
                sim_1_1[r, :] = -1; sim_1_1[:, c:c+2] = -1


        elif best_type == '2-1': # Src r, r+1 vs Tgt c
            if not used_src[r] and not used_src[r+1] and not used_tgt[c]:
                results.append({'src': src_sents[r]+src_sents[r+1], 'tgt': tgt_sents[c], 'type': '2-1'})
                used_src[r:r+2] = True; used_tgt[c] = True
                valid_match = True
                sim_2_1[r, c] = -1
                sim_1_1[r:r+2, :] = -1; sim_1_1[:, c] = -1

        elif best_type == '1-3': # Src r vs Tgt c, c+1, c+2
            if not used_src[r] and not used_tgt[c] and not used_tgt[c+1] and not used_tgt[c+2]:
                results.append({'src': src_sents[r], 'tgt': tgt_sents[c]+" "+tgt_sents[c+1]+" "+tgt_sents[c+2], 'type': '1-3'})
                used_src[r] = True; used_tgt[c:c+3] = True
                valid_match = True
                sim_1_3[r, c] = -1
                sim_1_1[r, :] = -1; sim_1_1[:, c:c+3] = -1

        elif best_type == '3-1': # Src r, r+1, r+2 vs Tgt c
            if not used_src[r] and not used_src[r+1] and not used_src[r+2] and not used_tgt[c]:
                results.append({'src': src_sents[r]+src_sents[r+1]+src_sents[r+2], 'tgt': tgt_sents[c], 'type': '3-1'})
                used_src[r:r+3] = True; used_tgt[c] = True
                valid_match = True
                sim_3_1[r, c] = -1
                sim_1_1[r:r+3, :] = -1; sim_1_1[:, c] = -1

        # Nếu không match được (do conflict đã bị dùng) -> xóa điểm này để tìm max tiếp theo
        if not valid_match:
            if best_type == '1-1': sim_1_1[r, c] = -1
            elif best_type == '1-2': sim_1_2[r, c] = -1
            elif best_type == '2-1': sim_2_1[r, c] = -1
            elif best_type == '1-3': sim_1_3[r, c] = -1
            elif best_type == '3-1': sim_3_1[r, c] = -1

        # Cập nhật kết quả
        if valid_match:
            results[-1]['score'] = float(best_score)

    return results

In [ ]:
device = get_device()
print(f"Đang tải model LaBSE trên {device} (n=3)...")
model = SentenceTransformer('sentence-transformers/LaBSE', device=device)

df = pd.read_csv(INPUT_FILE)
df["src_lang"] = df["src_lang"].fillna("").astype(str)
df["tgt_lang"] = df["tgt_lang"].fillna("").astype(str)

final_data = []

print("Bắt đầu Alignment (1-1, 1-2, 2-1, 1-3, 3-1)...")
for idx, row in tqdm(df.iterrows(), total=len(df)):
    src_sents = split_sentences(row.get('src_lang', ''), lang='zh')
    tgt_sents = split_sentences(row.get('tgt_lang', ''), lang='vi')
    pairs = align_n3(src_sents, tgt_sents, model, device, ALIGN_THRESHOLD)

    for p in pairs:
        final_data.append({
            'src_id': row.get('src_id', ''),
            'src_lang': p['src'],
            'tgt_lang': p['tgt'],
            'score': p['score'],
            'type': p.get('type', '1-1')
        })

# Thống kê báo cáo
df_res = pd.DataFrame(final_data)
type_counts = df_res['type'].value_counts()

print("Kết quả Alignment")
print(f"Tổng số cặp: {len(df_res)}")
print(f"Phân bố cặp câu: {type_counts}")
print(f"Average Semantic Score: {df_res['score'].mean():.4f}")

# Lưu file
df_res[['src_id', 'src_lang', 'tgt_lang']].to_csv(OUTPUT_FILE, index=False, encoding='utf-8-sig')
print(f"Đã lưu file: {OUTPUT_FILE}")

Đang tải model LaBSE trên cpu (n=3)...
Bắt đầu Alignment (1-1, 1-2, 2-1, 1-3, 3-1)...


100%|██████████| 1163/1163 [09:06<00:00,  2.13it/s]

Kết quả Alignment
Tổng số cặp: 1682
Phân bố cặp câu: type
1-1    1501
2-1     110
1-2      64
3-1       4
1-3       3
Name: count, dtype: int64
Average Semantic Score: 0.8445
Đã lưu file: ../Alignment/JSON2_5816_6978.csv


6. Evaluate của Alignment (json 2)

In [ ]:
# Config
SRC = 'JSON2'
FROM_ID = 5816
TO_ID = 6978
INPUT_FILE = f"../Alignment/{SRC}_{FROM_ID}_{TO_ID}.csv"
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'

In [ ]:
def get_proxy_translations(df, sample_size):
    """
    Dịch câu nguồn tiếng Trung sang tiếng Việt dùng Google Translate
    để làm tham chiếu so sánh.
    """
    print(f"Đang lấy mẫu ngẫu nhiên {sample_size} cặp câu để đánh giá...")

    # Lấy mẫu ngẫu nhiên
    sample = df.sample(n=min(sample_size, len(df)), random_state=10).copy()

    translator = Translator()
    hypotheses = [] # Câu trong file
    references = [] # Câu dịch máy (Làm chuẩn so sánh)
    success_count = 0

    for idx, row in tqdm(sample.iterrows(), total=len(sample)):
        src = str(row['src_lang'])
        tgt = str(row['tgt_lang']) # Đây là câu tiếng Việt gốc trong dataset

        # Bỏ qua câu quá ngắn hoặc rỗng
        if len(src) < 2 or len(tgt) < 5:
            continue

        try:
            # Dịch Trung -> Việt
            translated = translator.translate(src, src='zh-cn', dest='vi').text

            # Câu Google dịch là Reference (Tham chiếu) và câu Tgt gốc là Hypothesis (Giả thuyết)
            references.append(translated)
            hypotheses.append(tgt)

            success_count += 1
            time.sleep(0.5)

        except Exception as e:
            # Nếu lỗi mạng thì bỏ qua
            continue

    print(f"Đã dịch thành công {success_count} câu.")
    return hypotheses, references

def calculate_metrics(hypotheses, references):
    """Tính toán BLEU và BERTScore"""
    if not hypotheses:
        print("Không có dữ liệu để tính toán.")
        return

    # Tính BLEU Score
    # BLEU đánh giá độ khớp từ vựng (n-gram overlap)
    bleu = BLEU()
    bleu_score = bleu.corpus_score(hypotheses, [references])

    print(f"BLEU Score: {bleu_score.score:.2f}")

    # Tính BERTScore
    # BERTScore đánh giá độ tương đồng ngữ nghĩa (Semantic Similarity) dùng mô hình BERT
    P, R, F1 = score(hypotheses, references, lang='vi', verbose=False, device=DEVICE)

    avg_bert = F1.mean().item()
    print(f"BERTScore (F1): {avg_bert:.4f}")

    print("Ví dụ tham chiếu (Top 3):")
    for i in range(min(3, len(hypotheses))):
        print(f"Cặp {i+1}:")
        print(f"Dataset Tgt: {hypotheses[i]}")
        print(f"Google Trans: {references[i]}")

In [ ]:
df = pd.read_csv(INPUT_FILE)
# Thực hiện quy trình đánh giá
SAMPLE_SIZE = int(df.shape[0] * 0.1)  # Số lượng câu lấy mẫu để đánh giá
hyps, refs = get_proxy_translations(df, SAMPLE_SIZE)
# Tính điểm
calculate_metrics(hyps, refs)

Đang lấy mẫu ngẫu nhiên 168 cặp câu để đánh giá...


100%|██████████| 168/168 [05:57<00:00,  2.13s/it]


Đã dịch thành công 166 câu.
BLEU Score: 45.58
BERTScore (F1): 0.9011
Ví dụ tham chiếu (Top 3):
Cặp 1:
Dataset Tgt: Bởi vì tôi thích nóng, tôi thích nóng và ẩm ướt, và họ nói bạn sống ở nơi thích hợp.
Google Trans: Vì tôi thích nóng, thích nóng ẩm, người ta nói bạn sống đúng nơi.
Cặp 2:
Dataset Tgt: và tôi sẽ làm khám nghiệm tử thi nhưng tôi thích kính hiển vi, tôi nghĩ đó sẽ là lý do, nhưng ờ
Google Trans: Tôi định khám nghiệm tử thi nhưng tôi thích kính hiển vi và tôi nghĩ đó là lý do nhưng ồ
Cặp 3:
Dataset Tgt: Công dụng của nó là ngăn ngừa rụng trứng, nó cũng thay đổi cách thức mà chất nhầy cổ tử cung trông như thế nào và do đó chúng tôi đã có thể thấy rằng chúng tôi đã
Google Trans: Nó hoạt động bằng cách ngăn chặn sự rụng trứng và cũng làm thay đổi hình thức của chất nhầy cổ tử cung để chúng ta có thể biết rằng mình đã có
